# Mobility-Station-Finder

## Imports

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from matrixconverters.read_ptv import ReadPTVMatrix

## Read and process static data

### Paths

In [ ]:
path_npvm_zones = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\Verkehrszonen_Schweiz_NPVM_2017_shp.zip"
path_mobility_stations = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\mobility-stationen-und-fahrzeuge-schweiz.csv"
path_pt_skims = r"TODO"

### Read NPVM-zones

In [ ]:
df_npvm_zones = gpd.read_file(path_npvm_zones).to_crs(4326)

In [ ]:
df_npvm_zones.head()

### Read an process Mobility-stations

In [ ]:
df_mobility = pd.read_csv(path_mobility_stations, delimiter=";", encoding="utf8")[["Stationsnummer", "Name", "Standort"]].groupby("Stationsnummer").first().dropna()

In [ ]:
df_mobility["lon"] = df_mobility["Standort"].apply(lambda x: x.split(",")[1])
df_mobility["lat"] = df_mobility["Standort"].apply(lambda x: x.split(",")[0])

In [ ]:
df_mobility

In [ ]:
gdf_mobility = gpd.GeoDataFrame(df_mobility, geometry=geopandas.points_from_xy(df_mobility.lon, df_mobility.lat), crs=4326)

In [ ]:
gdf_mobility.head()

### Assign NPVM-zone to Mobility-Stations

In [ ]:
mobility_zone = gpd.sjoin(gdf_mobility, df_npvm_zones)

In [ ]:
mobility_zone.to_csv(r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\Analyse_Mobility_Stationen.csv", sep=";")

## Execute query

### Define and process origin and destination coordinates

In [ ]:
orig_point = Point(7.423570, 46.936620)
dest_point = Point(7.695260, 46.828540)

In [ ]:
df_orig = geopandas.GeoDataFrame({'geometry': [orig_point]}, crs="EPSG:4326")
df_dest = geopandas.GeoDataFrame({'geometry': [dest_point]}, crs="EPSG:4326")

### Assign NPVM-zone to origin and destination

In [ ]:
orig_zone = gpd.sjoin(df_orig, df_npvm_zones)
dest_zone = gpd.sjoin(df_dest, df_npvm_zones)

In [ ]:
orig_zone

In [ ]:
dest_zone

### Visualize situation on map

In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import geopandas
import json


m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = df_npvm_zones,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

m.add_layer(geo_data)
m.add_control(LayersControl())

In [ ]:
m.save("npvm-zonen.html")